In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

This module was written to set up and manipulate cameras (`'bpy.types.Camera'`), scenes (`'bpy.context.scene'`) and other things in Blender from Jupyter without opening it.

This notebook **does not use** a Python kernel, [uses a Blender kernel](https://github.com/panzi/blender_ipython).

In [ ]:
import bpy
import os
from PIL import Image
import pyexiv2
import math

In [ ]:
def open_blend(blendfile):
    """Opens a Blender file to be used with scripts."""
    try:
        bpy.ops.wm.open_mainfile(filepath=blendfile)
        print('Opened:', blendfile)
    except IOError:
        print('No such file:', blendfile)

In [ ]:
def set_up_sc(**kwargs):
    """Set up a scene ('bpy.context.scene') and returns it."""
    wpx = kwargs.pop('wpx', 320)  # Capture width in pixels.
    hpx = kwargs.pop('hpx', 240)  # Capture hight in pixels.
    antialias = kwargs.pop('antialias', '8')  # ('5', '8', '11', '16')
    bw = kwargs.pop('bw', True)  # True for capture in grayscale.
    cam = kwargs.pop('cam', 'iph4s')
    scalepercent = kwargs.pop('scalepercent', 100)
    ext = kwargs.pop('ext', 'TIFF')  # Extension of the ouput file.
    transp = kwargs.pop('transp', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(set_up_sc.__name__,
                  list(kwargs.keys())[-1]))
    sc = bpy.context.scene
    if transp:
        sc.render.image_settings.file_format = 'PNG'
        sc.render.alpha_mode = 'TRANSPARENT'
        sc.render.image_settings.color_mode = 'RGBA'
    else:  # 'RGBA' is not supported by JPEG files.
        sc.render.image_settings.file_format = ext
        sc.world.horizon_color = (1, 1, 1)
        if bw:
            sc.render.image_settings.color_mode = 'BW'
        else:
            sc.render.image_settings.color_mode ='RGB'
    sc.render.use_stamp_lens = True
    sc.render.resolution_percentage = scalepercent
    sc.render.resolution_x = wpx
    sc.render.resolution_y = hpx
    sc.render.antialiasing_samples = antialias
    sc.render.use_overwrite = True
    return sc

In [ ]:
def get_camera():
    """Returns the first camera ('bpy.types.Camera') of the current
    Blender file."""
    for i in bpy.data.objects:
        if i.type == 'CAMERA':
            return i

In [ ]:
def get_mesh(name, **kwargs):
    """Makes a 'PLAIN_AXEXS' parent of an object. name (str) is
    the name of the child."""
    loc0 = kwargs.pop('loc0', (0, 0, 0))  # Initial location of the
                                          # axes. 
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_mesh.__name__,
                  list(kwargs.keys())[-1]))
    bpy.ops.object.empty_add(type='PLAIN_AXES', location=loc0)
    axis = bpy.context.active_object
    axis.name = 'axis'
    for obj in bpy.data.objects:
        if obj.name == name:
            obj.parent = axis
            return axis, obj
    return -1

In [ ]:
def set_up_cam(cam, **kwargs):
    """Set up a camera ('bpy.types.Camera') as a real camera.
    
    Add more camera presets to d_cams ('dict') transcribing the values
    from the files in /usr/share/blender/scripts/presets/camera/ to
    the d_cams (keyword argument of type dictt)."""
    idcam = kwargs.pop('idcam', 'iph4s')
    loc = kwargs.pop('loc', (0, 0, 0))
    rot = kwargs.pop('rot', (0, 0, 0))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(set_up_cam.__name__,
                  list(kwargs.keys())[-1]))
    d_cams = {'iph4s': ['iPhone 4S', 4.54, 3.42, 4.28, 'HORIZONTAL'], }
    cam.location = loc
    for i in range(len(rot)):
        cam.rotation_euler[i] = rot[i]
    cam.data.sensor_width = d_cams[idcam][1]
    cam.data.sensor_height = d_cams[idcam][2]
    cam.data.lens = d_cams[idcam][3]
    cam.data.sensor_fit = d_cams[idcam][4]

In [ ]:
def num_str_zeros(num, n_digs, firstis1=False):
    """Returns a string that contains a sequence of n_digs - len(num)
    zeros followed by num (int).
    
    Args:
        num: A non-negative integer number (int).
        n_digs: The length of the string that will contains zeros and
            num (int).
        firstis1: Plus 1 to num if it is True. It is useful for
            filenames.
            
    From fwdimaging.ipynb (Python kernel).
    
    >>> num_str_zeros(89, 5)
    '00089'
    >>> num_str_zeros(0, 4, True)
    '0001'
    """
    if num < 0:
        num = 0
    if firstis1:
        num += 1
    len_num = len(str(int(num)))
    str_num = ''
    for i in range(n_digs - len_num):
        str_num += '0'
    str_num += str(int(num))
    return str_num

In [ ]:
def find_out_dir(**kwargs):
    """Find out a directory and returns the route.
    
    With replace (bool) equal to False, this will create a new
    directory if dirname (string) exists in <s3out>/<parentdir>/"""
    s3out = kwargs.pop('s3out',
                       os.path.join(os.path.expanduser('~'),
                                    's3-out'))
    dirname = kwargs.pop('dirname', 'blend-phg-set-0001')
    parentdir = kwargs.pop('parentdir', 'scanner')
    path = kwargs.pop('path', os.path.join(s3out, parentdir, dirname))
    replace = kwargs.pop('replace', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(find_out_dir.__name__,
                  list(kwargs.keys())[-1]))
    if not replace:
        # Create a new direcctory with a different name.
        while True:
            if not os.path.exists(path):
                print('Make directory:', path)
                os.makedirs(path)
                break
            else:
                str_num = path.split('-')[-1]
                num_dir = int(str_num)
                num_dir += 1
                str_num_dir = num_str_zeros(num_dir, len(str_num))
                path = ('-'.join(path.split('-')[:-1]) + '-'
                        + str_num_dir)
    return path

In [ ]:
def copy_exif(dest_path, src_path):
    """Copies the Exif metadata from a source image in src_path (str)
    to another in dest_path (str)."""
    dest_img = Image.open(dest_path)
    wpx = dest_img.size[0]
    hpx = dest_img.size[1]
    dest_img.close()
    src_img = pyexiv2.ImageMetadata(src_path)
    src_img.read()
    dest_img = pyexiv2.ImageMetadata(dest_path)
    dest_img.read()
    src_img.copy(dest_img, exif=True)
    dest_img["Exif.Photo.PixelXDimension"] = wpx
    dest_img["Exif.Photo.PixelYDimension"] = hpx
    dest_img.write()

In [ ]:
def shoot_cam(cam, sc, n_photo, len_img_set, path, **kwargs):
    """Renders a scena (sc, 'bpy.context.scene') with a camera
    (cam, 'bpy.types.Camera') and saves the result in paht.
    
    Args:
        n_photo: A number of a photo in set (int).
        len_img_set: The number of photos (int) that will contain the
            image set.
    """
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'superscanner-software-s3'))
    prefix = kwargs.pop('prefix', 'view_')
    exif = kwargs.pop('exif',
                      os.path.join(s3path,
                                   'img',
                                   'Photo 25-09-16 11 11 00.jpg'))
    base_file_name = prefix + num_str_zeros(n_photo,
                                            len(str(len_img_set)),
                                            firstis1=True)
    base_file_path = os.path.join(path, base_file_name)
    separatedir = kwargs.pop('separatedir', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(shoot_cam.__name__,
                  list(kwargs.keys())[-1]))
    if separatedir and not os.path.exists(base_file_path):
        print('Make directory:', base_file_path)
        os.mkdir(base_file_path)
        base_file_path = os.path.join(base_file_path, os.path.basename(base_file_path))
    # sc.render.filepath does not require the explicit extension.
    sc.render.filepath = base_file_path
    # The next line will save the rendered scene with .tif
    # extension.
    bpy.ops.render.render(write_still=True)
    # The file format is chosen in set_up_sc().
    ext = sc.render.image_settings.file_format
    ext = ext.lower()
    if ext == 'jpeg':
        ext = 'jpg'
    elif ext == 'tiff':
        ext = 'tif'
    file_path = base_file_path + '.' + ext
    # PNG and TIFF/TIF files does not support Exif metadata.
    if ext == 'png' or ext == 'tif':
        print('Saved view without Exif metadata:', file_path)
    else:
        copy_exif(file_path, exif)
        print('Saved view with Exif metadata:', file_path)

In [ ]:
def put_mesh(path, l_locs, **kwargs):
    """Imports and clones (if keyword argument 'copies' is greater
    than 1) a STL mesh from path (str), and locates it and its copies
    in the the (x, y, z) coordinates of l_locs (list)."""
    copies = kwargs.pop('copies', 1)  # Number of copies of the mesh.
    # The majority of SuperScanner's parts must be scalated to 0.001
    # in Blender.
    scale = kwargs.pop('scale', 0.001)
    rotation = kwargs.pop('rotation', (0, 0, 0))
    inrad = kwargs.pop('inrad', False)  # Are the rotation angles
                                        # expressend in radians? 
    parent = kwargs.pop('parent', None)  # The name in Blender of the
                                         # parent object. 
    offset = kwargs.pop('offset', (0, 0, 0))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(put_mesh.__name__,
                  list(kwargs.keys())[-1]))
    bpy.ops.object.select_all(action='DESELECT')
    # Impor the STL mesh.
    bpy.ops.import_mesh.stl(filepath=path, global_scale=scale)
    i = 0
    while True:
        if isinstance(l_locs[i], tuple):
            clone = bpy.context.scene.objects.active
            rel_loc_clone = l_locs[i]  # Clone's relative positione
                                       # to the parent. If parent
                                       # does not exist, rel_loc_clones
                                       # will be an absolute location. 
            if parent:
                parent_obj = bpy.data.objects[parent]
                clone.parent = parent_obj
            for j in range(3):
                if inrad:
                    clone.rotation_euler[j] = rotation[j]
                else:
                    clone.rotation_euler[j] = math.radians(rotation[j])
                clone.location[j] = rel_loc_clone[j] + offset[j]
        if i >= copies - 1:
            break
        else:
            i += 1
            bpy.ops.object.duplicate()